In [15]:
import sagemaker
import os

from tqdm import tqdm
from sagemaker.huggingface.model import HuggingFacePredictor
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [4]:
predictor=HuggingFacePredictor(
    endpoint_name='huggingface-pytorch-inference-2022-04-06-09-35-08-888',
)

In [6]:
import pandas as pd
data = pd.read_excel('./data/more_test.xlsx')
data.columns

Index(['原句', '手动改写', 'SC改写', 'Quillbot改写'], dtype='object')

In [16]:
profix=['Low leval paraphrase:','Medium leval paraphrase:','High leval paraphrase:']
res={'Low leval paraphrase:':[], 'Medium leval paraphrase:':[], 'High leval paraphrase:':[]}
for i in profix:
    print(i)
    for j in tqdm(range(len(data))):
        out=predictor.predict({
            'inputs': i+data.loc[j,'原句'],
            "parameters": {"max_length": 256},
        })
        res[i].append(out[0]['generated_text'])

Low leval paraphrase:


100%|██████████| 167/167 [11:46<00:00,  4.23s/it]


Medium leval paraphrase:


100%|██████████| 167/167 [11:25<00:00,  4.10s/it]


High leval paraphrase:


100%|██████████| 167/167 [12:06<00:00,  4.35s/it]


In [17]:
with open("outputs/more-ref.txt","a+") as f:
     for i in range(len(data)):
            f.write(data.loc[i,'原句'])
            f.write('\n')
with open("outputs/more-quillbot.txt","a+") as f:
     for i in range(len(data)):
            f.write(data.loc[i,'Quillbot改写'])
            f.write('\n')
with open("outputs/more-sc.txt","a+") as f:
     for i in range(len(data)):
            f.write(data.loc[i,'SC改写'])
            f.write('\n')
with open("outputs/more-people.txt","a+") as f:
     for i in range(len(data)):
            f.write(data.loc[i,'手动改写'])
            f.write('\n')
with open("outputs/more-t5-low.txt","a+") as f:
     for i in range(len(data)):
            f.write(res['Low leval paraphrase:'][i])
            f.write('\n')
with open("outputs/more-t5-medium.txt","a+") as f:
     for i in range(len(data)):
            f.write(res['Medium leval paraphrase:'][i])
            f.write('\n')
with open("outputs/more-t5-high.txt","a+") as f:
     for i in range(len(data)):
            f.write(res['High leval paraphrase:'][i])
            f.write('\n')


In [ ]:
!pip install git+https://github.com/Maluuba/nlg-eval.git@master
!nlg-eval --setup

In [25]:
from nlgeval import compute_metrics
metrics_dict = compute_metrics(hypothesis='outputs/more-t5-high.txt',
                               references=['outputs/more-ref.txt'])
metrics_dict

Bleu_1: 0.370252
Bleu_2: 0.267915
Bleu_3: 0.206425
Bleu_4: 0.162956
METEOR: 0.260194
ROUGE_L: 0.330744
CIDEr: 1.115817
SkipThoughtsCosineSimilarity: 0.828887
EmbeddingAverageCosineSimilarity: 0.912593
EmbeddingAverageCosineSimilairty: 0.912593
VectorExtremaCosineSimilarity: 0.626925
GreedyMatchingScore: 0.769654


{'Bleu_1': 0.37025215448440146,
 'Bleu_2': 0.26791507195691455,
 'Bleu_3': 0.20642478359330174,
 'Bleu_4': 0.16295606204511567,
 'METEOR': 0.2601943874979247,
 'ROUGE_L': 0.33074430572482444,
 'CIDEr': 1.115816690717915,
 'SkipThoughtCS': 0.8288867,
 'EmbeddingAverageCosineSimilarity': 0.912593,
 'EmbeddingAverageCosineSimilairty': 0.912593,
 'VectorExtremaCosineSimilarity': 0.626925,
 'GreedyMatchingScore': 0.769654}